In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
from sql_db_functions import sql_db_functions as SQLf

import pandas as pd

In [2]:
conn, cursor = SQLf.connect_sql()

In [3]:
query = f"SELECT product_characteristics.id, Brand_id, Detail,Summary, Brand, Price, Link FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
#rows

In [4]:
conn, cursor = SQLf.connect_sql()

In [5]:
query = f"SELECT Brand_id, image_link, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows

In [6]:
SQLf.close_connection_db(conn=conn,cursor=cursor)

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(rows ,columns= ['id', 'Brand_id', 'Detail','Summary', 'Brand', 'Price', 'Link'])
df.head()

,id,Brand_id,Detail,Summary,Brand,Price,Link
0,2,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The shirt depicted in the images is a Gucci Ox...,Gucci,650.0,https://www.gucci.com/it/it/pr/women/ready-to-...
1,4,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The showcased shirt is a luxurious Gucci creat...,Gucci,1300.0,https://www.gucci.com/it/it/pr/women/ready-to-...
2,6,6,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The shirt showcased in the images is a Gucci p...,Gucci,1100.0,https://www.gucci.com/it/it/pr/women/ready-to-...
3,8,8,*Type: Top\n*Fit: Regular fit\n*Sleeve style: ...,The showcased top from Gucci is a floral lace ...,Gucci,980.0,https://www.gucci.com/it/it/pr/women/ready-to-...
4,11,10,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,"The Gucci cotton poplin shirt, inspired by sum...",Gucci,850.0,https://www.gucci.com/it/it/pr/women/ready-to-...


In [9]:
df_img = pd.DataFrame(rows_img ,columns= ['Brand_id','image_link','base64'])
df_img['image_link'] = df_img['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )
df_img.head()

,Brand_id,image_link,base64
0,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,1,https://media.gucci.com/style/White_Center_0_0...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,2,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [11]:
df.to_csv('main_0.6.csv')
df_img.to_csv('images_0.6.csv')

In [192]:
df['Detail'] = df['Detail'] = df['Detail'].str.replace('\n', ' / ').str.replace(r'\s+', ' ').str.replace('*', ' ')

In [193]:
import numpy as np

In [194]:
final_df = pd.DataFrame()

for parent_row in df.Detail: 
    x = []
    for row in parent_row.split('/'):
        x.append(row.split(':'))

    temp_dataframe = pd.DataFrame(x)
    temp_dataframe[0] = temp_dataframe[0].str.lower()
    temp_dataframe = temp_dataframe.sort_values(by=0)
    temp_dataframe = temp_dataframe.groupby(0)[1].apply(lambda x: ', '.join(x)).reset_index()
        
    temp_dataframe = temp_dataframe.set_index(0).T
    temp_dataframe.columns = temp_dataframe.columns.str.lstrip()
    
    final_df = pd.concat([final_df,temp_dataframe], ignore_index = True)

final_df['seasson'] = final_df['season'].combine_first(final_df['seasson'])
final_df = final_df.drop('season', axis=1)

In [214]:
final_df.head(2)

,breathability,collar style,colors,details and embellishments,fabric weight,fit,fluidity of fabric,formality,lapel,length,...,patterns placement,pocket placement,pocket presence,pocket size,shape,sleeve style,texture,transparency,type,seasson
0,High,Button-down,"80% Pink, 20% Light Pink",Buttons,Light,Regular fit,High,Formal,Not applicable,Hip-length,...,Not applicable,Not applicable,No,Not applicable,Fitted,"Long , Long sleeve",Smooth,Semi-transparent,Shirt,"Spring, Summer"
1,High,Button-down,Light Blue 100%,"Buttons, Embroidery",Light,Regular fit,Medium,"Business Casual, Casual",Not applicable,Hip-length,...,All over,Not applicable,No,Not applicable,Fitted,"Long , Long sleeve",Smooth,Opaque,Shirt,"Spring, Summer, Autumn"


In [219]:
df = pd.concat([df,final_df], axis=1)

In [221]:
df.head(1)

,id,Brand_id,Detail,Summary,Brand,Price,Link,breathability,collar style,colors,...,patterns placement,pocket placement,pocket presence,pocket size,shape,sleeve style,texture,transparency,type,seasson
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve...,The Gucci silk shirt features the iconic GG mo...,Gucci,1500.0,https://www.gucci.com/it/it/pr/women/ready-to-...,High,Button-down,"80% Pink, 20% Light Pink",...,Not applicable,Not applicable,No,Not applicable,Fitted,"Long , Long sleeve",Smooth,Semi-transparent,Shirt,"Spring, Summer"


In [223]:
df.columns

Index(['id', 'Brand_id', 'Detail', 'Summary', 'Brand', 'Price', 'Link',
       'breathability', 'collar style', 'colors', 'details and embellishments',
       'fabric weight', 'fit', 'fluidity of fabric', 'formality', 'lapel',
       'length', 'material', 'neckline', 'occasion suitability', 'patterns',
       'patterns placement', 'pocket placement', 'pocket presence',
       'pocket size', 'shape', 'sleeve style', 'texture', 'transparency',
       'type', 'seasson'],
      dtype='object')

In [206]:
import matplotlib.pyplot as plt

In [224]:
df.to_csv('main.csv')

In [225]:
df_img.to_csv('img.csv')